In [3]:
"""

@author:Blesson George
"""

'\n\n@author:Blesson George\n'

In [4]:
# import necessary packages
import tensorflow as tf
import json
from keras.models import Sequential
from tf.keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
import numpy as np

Using TensorFlow backend.


In [0]:
def balanced_sample_maker(X, y, sample_size, random_seed=42):
    uniq_levels = np.unique(y)
    uniq_counts = {level: sum(y == level) for level in uniq_levels}

    if not random_seed is None:
        np.random.seed(random_seed)

    # find observation index of each class levels
    groupby_levels = {}
    for ii, level in enumerate(uniq_levels):
        obs_idx = [idx for idx, val in enumerate(y) if val == level]
        groupby_levels[level] = obs_idx
    # oversampling on observations of each label
    balanced_copy_idx = []
    for gb_level, gb_idx in groupby_levels.items():
        over_sample_idx = np.random.choice(gb_idx, size=sample_size, replace=True).tolist()
        balanced_copy_idx+=over_sample_idx
    np.random.shuffle(balanced_copy_idx)
    data_train=X[balanced_copy_idx]
    labels_train=y[balanced_copy_idx]
    if  ((len(data_train)) == (sample_size*len(uniq_levels))):
        print('number of sampled example ', sample_size*len(uniq_levels), 'number of sample per class ', sample_size, ' #classes: ', len(list(set(uniq_levels))))
    else:
        print('number of samples is wrong ')
    print('data train',data_train.shape, labels_train)	
    return (data_train,labels_train,balanced_copy_idx)

def next_picker(nX,ny,old_train,old_label,incorrect,prob):
    x=nX[incorrect]
    y=ny[incorrect]
    uniq_levels=np.unique(y)
    uniq_counts={lev:sum(y==lev) for lev in uniq_levels}
    groupby_levels = {}
    for ii, level in enumerate(uniq_levels):
        obs_idx = [idx for idx, val in enumerate(y) if val == level]
        groupby_levels[level] = obs_idx
    # oversampling on observations of each label
    balanced_copy_idx = []
    for gb_level, gb_idx in groupby_levels.items():
        #print(type(gb_idx),len(gb_idx),gb_idx)
        #print([k for k in gb_idx])
        #print(prob.keys())
        prob_lev={k:prob[k] for k in gb_idx}
        
        maxx=max(prob_lev.values())
        keys = [x for x,y in prob_lev.items() if y ==maxx]
        if len(keys)>1:
            keys=[(np.random.choice(np.array(keys)))]
            
        balanced_copy_idx+=keys
    print('y',y)
    print('balanced copy',balanced_copy_idx)
    data_train=x[balanced_copy_idx]
    labels_train=np.reshape(y[balanced_copy_idx],(10,1))
    newX =nX[np.setdiff1d(np.arange(nX.shape[0]), balanced_copy_idx)]
    newy = ny[np.setdiff1d(np.arange(ny.shape[0]), balanced_copy_idx)]
    print('old train',old_train.shape,'data train',data_train.shape)
    print('old label',old_label.shape,'labels train',labels_train.shape)
    data_train=np.concatenate((old_train,data_train),axis=0)
    labels_train=np.concatenate((old_label,labels_train),axis=0)
    return (data_train,labels_train,newX,newy)

 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

In [6]:
batch_size=4
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print(x_train.shape,x_test.shape,type(x_train),y_train.shape,y_test.shape)
data=np.concatenate((x_train,x_test),axis=0)
print('DATA',data.shape)

170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3) (10000, 32, 32, 3) <class 'numpy.ndarray'> (50000, 1) (10000, 1)
DATA (60000, 32, 32, 3)


In [7]:
labels=np.concatenate((y_train,y_test),axis=0)
#labels=labels[0:100]
print('LABELS',labels.shape)

LABELS (60000, 1)


In [0]:
#z-score
mean = np.mean(data,axis=(0,1,2,3))
std = np.std(data,axis=(0,1,2,3))
data=(data-mean)/std+1e-7

In [13]:
num_classes = 10

weight_decay = 1e-4
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.summary()
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)       

In [21]:
#Train trainset and labelset
trainset,labelset,sample_idx=balanced_sample_maker(data,labels,1)
newX = data[np.setdiff1d(np.arange(data.shape[0]), sample_idx)]
newy = labels[np.setdiff1d(np.arange(labels.shape[0]), sample_idx)]


number of sampled example  10 number of sample per class  1  #classes:  10
data train (10, 32, 32, 3) [[0]
 [6]
 [9]
 [1]
 [8]
 [5]
 [2]
 [3]
 [4]
 [7]]


In [22]:
import tensorflow as tf 
import os
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)

tpu_model.compile(
    optimizer=tf.train.RMSPropOptimizer(learning_rate=0.001,decay=1e-6),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)


W0715 13:14:53.501694 140487790315392 keras_support.py:217] Keras support is now deprecated in support of TPU Strategy. Please follow the distribution strategy guide on tensorflow.org to migrate to the 2.0 supported version.
W0715 13:15:02.273197 140487790315392 keras_support.py:1394] Keras support is now deprecated in support of TPU Strategy. Please follow the distribution strategy guide on tensorflow.org to migrate to the 2.0 supported version.


In [0]:


model.save_weights('model.h5')
iter=0
results={}
for iter in range(0,1000):
    print('Iteration No',iter,'starts')    
    print('Trainset Shape',trainset.shape,'Trainlabels',labelset.shape)
    datagen.fit(trainset)
    label_one_hot=np_utils.to_categorical(labelset,num_classes)
    model.fit_generator(datagen.flow(trainset, label_one_hot, batch_size=4),\
                    steps_per_epoch=trainset.shape[0] //batch_size,epochs=125,\
                    verbose=2,callbacks=[LearningRateScheduler(lr_schedule)])#,EarlyStopping(monitor='val_loss', patience=2, verbose=2,mode='auto',baseline=None)])#,restore_best_weights=True)])
    #save to disk
    #model_json = model.to_json()
    #with open('model.json', 'w') as json_file:
     #   json_file.write(model_json)

    #model.save_weights('model.h5')  
    ans=model.predict(newX,batch_size=32, verbose=2)
    print('prediction',ans.shape,type(ans))  
    maxlabel=np.argmax(ans,axis=1)
    #maxlabel=np.array(maxlabel.reshape(maxlabel.shape[0],))
    print('MaxLabel',maxlabel.shape)
    maxval=np.max(ans,axis=1)
    print('Maxval',maxval.shape)
    newy=np.array(newy.reshape(newy.shape[0],))
    print('newy',newy.shape)
    print('maxval',maxval)
    print('maxlabel',maxlabel)
    print('newy',newy)

    incorrect=np.where(maxlabel!=newy)
    print(incorrect)
    newyset=np_utils.to_categorical(newy,num_classes)
    scores = model.evaluate(newX,newyset, batch_size=128, verbose=10)
    #print('\nIteration No:%i Test result: %.3f loss: %.3f' % (iter,scoresi[1]*100,scores[0]))
    results[iter]=('Accuracy:',scores[1]*100,'Loss:',scores[0],'Training Size:',trainset.shape[0])
    with open('resapr_30.json','w') as fp:
        json.dump(results,fp)   
    #    file.write(res)
    trainset,labelset,newX,newy=next_picker(newX,newy,trainset,labelset,incorrect,maxval)
    print('\nIteration No:%i Test result: %.3f loss: %.3f' % (iter,scores[1]*100,scores[0]))
    model.load_weights('model.h5')

In [0]:
# Clone the entire repo.
!git clone -l -s git://github.com/jakevdp/PythonDataScienceHandbook.git cloned-repo
%cd cloned-repo
!ls

In [0]:
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://api.github.com/repos/jakevdp/PythonDataScienceHandbook/contents/notebooks/data/california_cities.csv

In [0]:
"""
#    iter+=1
    for i in range(0,newX.shape[0]):
        if (np.argmax(model.predict(newX[i].reshape(1,32,32,3)),axis=1)!=newy[i]):

            incorrect.append(i)
        prob_list[i]=(np.max(model.predict(newX[i].reshape(1,32,32,3)),axis=1))
    print('Total No.',newX.shape[0],'InCorrect Predictions',len(incorrect))    
   ''' 
    newyset=np_utils.to_categorical(newy,num_classes)
    scores = model.evaluate(newX,newyset, batch_size=128, verbose=10)
    #print('\nIteration No:%i Test result: %.3f loss: %.3f' % (iter,scoresi[1]*100,scores[0]))
    results[iter]=('Accuracy:',scores[1]*100,'Loss:',scores[0],'Training Size:',trainset.shape[0])
    with open('resapr_26.json','w') as fp:
        json.dump(results,fp)   
    #    file.write(res)
    trainset,labelset,newX,newy=next_picker(newX,newy,trainset,labelset,incorrect,maxval)
    print('\nIteration No:%i Test result: %.3f loss: %.3f' % (iter,scores[1]*100,scores[0]))
    model.load_weights('model.h5')"""